In [29]:
import torch
from torch.utils.data import DataLoader
from dataset import SingleFileTestDataset
import numpy as np
from model import Model
from datetime import date
from esm_pretrained import ESM
import time
import sys
import os

In [30]:
dir_path = os.path.abspath('')
print(dir_path)

/cluster/home/t116517uhn/llps-v2


In [31]:
learning_rate = 1e-4
num_layers = 1
model_dim = 320
num_heads = 4
ff_dim = 320
random_seed = 69
batch_size = 5 # change?
dropout = 0.3
loss_function = torch.nn.BCELoss()
model_name = '2023-01-04_full_e200_lr-4_dropout-0.3'
path = dir_path + '/sample-model/'

In [32]:
model_name = '2023-01-04_full_e200_lr-4_dropout-0.3'
path = '/cluster/projects/kumargroup/luke/output/v2/'

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
my_model = Model(device, 1, 320, 4, 320, 0.3)
my_model.load_state_dict(torch.load(f'{path}{model_name}.pt', map_location=torch.device('cpu')))
my_model.eval()

activation = {}
def get_activation(name):
    def hook(model, input, output):
        if name in activation:
            curr = activation[name]
            activation[name] += [input[0]]
        else:
            activation[name] = [input[0]]
    return hook

my_model.encoder.register_forward_hook(get_activation('encoder'))
my_model.classifier.register_forward_hook(get_activation('classifier'))

In [41]:
dataset = SingleFileTestDataset(dir_path + '/data/test_8_short_seqs.csv')

In [44]:
for i in range(len(dataset)):
    print(dataset[i])

('MLPLSLLKTAQNHPMLVELKNGETYNGHLVSCDNWMNINLREVICTSRDGDKFWRMPECYIRGSTIKYLRIPDEIIDMVKEEVVAKGRGRGGLQQQKQQKGRGMGGAGRGVFGGRGRGGIPGTGRGQPEKKPGRQAGKQ', 1)
('MATKAVCVLKGDGPVQGIINFEQKESNGPVKVWGSIKGLTEGLHGFHVHEFGDNTAGCTSAGPHFNPLSRKHGGPKDEERHVGDLGNVTADKDGVADVSIEDSVISLSGDHCIIGRTLVVHEKADDLGKGGNEESTKTGNAGSRLACGVIGIAQ', 1)
('MSSEEGKLFVGGLNFNTDEQALEDHFSSFGPISEVVVVKDRETQRSRGFGFITFTNPEHASVAMRAMNGESLDGRQIRVDHAGKSARGTRGGGFGAHGRGRSYSRGGGDQGYGSGRYYDSRPGGYGYGYGRSRDYNGRNQGGYDRYSGGNYRDNYDN', 1)
('MSDQEAKPSTEDLGDKKEGEYIKLKVIGQDSSEIHFKVKMTTHLKKLKESYCQRQGVPMNSLRFLFEGQRIADNHTPKELGMEEEDVIEVYQEQTGGHSTV', 1)
('MSHSVKIYDTCIGCTQCVRACPTDVLEMIPWGGCKAKQIASAPRTEDCVGCKRCESACPTDFLSVRVYLWHETTRSMGLAY', 0)
('MTMNITSKQMEITPAIRQHVADRLAKLEKWQTHLINPHIILSKEPQGFVADATINTPNGVLVASGKHEDMYTAINELINKLERQLNKLQHKGEARRAATSVKDANFVEEVEEE', 0)
('MINMKVAISMDVDKISNSFEDCKYFLIVRIDDNEVKSTKVIFNDESGKKSIVKENVNAIICKNISEENYKKFSKKIEIYHAEGDDVDKNISLFIEGELSKISNP', 0)
('MTHDNKLQVEAIKRGTVIDHIPAQIGFKLLSLFKLTETDQRITIGLNLPSGEMGRKDLIKIENTFLSEDQVDQLALYAPQATVNRIDNYEVVG

In [9]:
data = [
('1','MLPLSLLKTAQNHPMLVELKNGETYNGHLVSCDNWMNINLREVICTSRDGDKFWRMPECYIRGSTIKYLRIPDEIIDMVKEEVVAKGRGRGGLQQQKQQKGRGMGGAGRGVFGGRGRGGIPGTGRGQPEKKPGRQAGKQ'),
('1','MATKAVCVLKGDGPVQGIINFEQKESNGPVKVWGSIKGLTEGLHGFHVHEFGDNTAGCTSAGPHFNPLSRKHGGPKDEERHVGDLGNVTADKDGVADVSIEDSVISLSGDHCIIGRTLVVHEKADDLGKGGNEESTKTGNAGSRLACGVIGIAQ'),
('1','MSSEEGKLFVGGLNFNTDEQALEDHFSSFGPISEVVVVKDRETQRSRGFGFITFTNPEHASVAMRAMNGESLDGRQIRVDHAGKSARGTRGGGFGAHGRGRSYSRGGGDQGYGSGRYYDSRPGGYGYGYGRSRDYNGRNQGGYDRYSGGNYRDNYDN')
]

In [59]:
print('================BATCH SIZE 1======================')
activation = {}
data1 = [data[0]]

print(data1)

with torch.no_grad():
    out = my_model(data1).detach().cpu()

print(out)
embeds1 = activation['encoder']
print(embeds1[0].size())
# print(embeds1)
features1 = activation['classifier']
# print(features1)


================BATCH SIZE 1======================
[('1', 'MLPLSLLKTAQNHPMLVELKNGETYNGHLVSCDNWMNINLREVICTSRDGDKFWRMPECYIRGSTIKYLRIPDEIIDMVKEEVVAKGRGRGGLQQQKQQKGRGMGGAGRGVFGGRGRGGIPGTGRGQPEKKPGRQAGKQ')]
tensor([0.5461])
torch.Size([1, 141, 320])


In [60]:
# batch size 2
activation = {}
data2 = [data[0],data[2]]
print(data2)
with torch.no_grad():
    out = my_model(data2).detach().cpu()

print(out)
embeds2 = activation['encoder']
print(embeds2[0].size())
features2 = activation['classifier']
# print(features2[0])


[('1', 'MLPLSLLKTAQNHPMLVELKNGETYNGHLVSCDNWMNINLREVICTSRDGDKFWRMPECYIRGSTIKYLRIPDEIIDMVKEEVVAKGRGRGGLQQQKQQKGRGMGGAGRGVFGGRGRGGIPGTGRGQPEKKPGRQAGKQ'), ('1', 'MSSEEGKLFVGGLNFNTDEQALEDHFSSFGPISEVVVVKDRETQRSRGFGFITFTNPEHASVAMRAMNGESLDGRQIRVDHAGKSARGTRGGGFGAHGRGRSYSRGGGDQGYGSGRYYDSRPGGYGYGYGRSRDYNGRNQGGYDRYSGGNYRDNYDN')]
tensor([[0.4947],
        [0.9913]])
torch.Size([2, 159, 320])


In [62]:
new_seq = ('1','MSSEEGKLFVGGLNFNTDEQALEDHFSSFGPISEVVVVKDRETQRSRGFGFITFTNPEHASVAMRAMNGESLDGRQIRVDHAGKSARGTRGGGFGAHGRGRSYSRGGGDQGYGSGRYYDSRPGGYGYGYGRSRDYNGRN')
activation = {}
data2 = [data[0],new_seq]
print(data2)
with torch.no_grad():
    out = my_model(data2).detach().cpu()

print(out)
embeds2 = activation['encoder']
print(embeds2[0].size())
features2 = activation['classifier']

[('1', 'MLPLSLLKTAQNHPMLVELKNGETYNGHLVSCDNWMNINLREVICTSRDGDKFWRMPECYIRGSTIKYLRIPDEIIDMVKEEVVAKGRGRGGLQQQKQQKGRGMGGAGRGVFGGRGRGGIPGTGRGQPEKKPGRQAGKQ'), ('1', 'MSSEEGKLFVGGLNFNTDEQALEDHFSSFGPISEVVVVKDRETQRSRGFGFITFTNPEHASVAMRAMNGESLDGRQIRVDHAGKSARGTRGGGFGAHGRGRSYSRGGGDQGYGSGRYYDSRPGGYGYGYGRSRDYNGRN')]
tensor([[0.5461],
        [0.9812]])
torch.Size([2, 141, 320])


In [63]:
torch.all(torch.eq(embeds1[0], embeds2[0][0,:,:]))

tensor(True)

In [48]:
activation = {}
dataloader5 = DataLoader(dataset, batch_size=5, shuffle=False)

outputs5 = torch.Tensor()
for data in iter(dataloader5):
    with torch.no_grad():
        x, y = data
        inputs = []
        for n in range(len(x)):
            inputs.append((y[n], x[n]))

        out = my_model(inputs).squeeze().detach().cpu()
        outputs5 = torch.cat((outputs5, out), dim=0)

embeds5 = activation['encoder']
print(embeds5[0][0,:,:])
features5 = activation['classifier']
print(features5[0][0,:,:])
print(outputs5)

tensor([[ 0.2306,  0.7535,  0.1205,  ...,  1.1824, -0.1949, -0.4452],
        [ 0.4109,  0.2706, -0.1165,  ...,  0.6309,  0.0900, -0.0671],
        [ 0.2159, -0.2961,  0.1187,  ...,  0.3290, -0.0442,  0.3566],
        ...,
        [-0.0486, -0.0532,  0.0735,  ...,  0.3742, -0.4053, -0.0050],
        [ 0.1651,  0.0862,  0.1011,  ..., -0.4449, -0.6215, -0.1898],
        [ 0.0190,  0.0915,  0.2531,  ...,  0.4676, -0.7663, -0.1630]])
tensor([[-0.2749,  1.5427,  0.4962,  ...,  2.0433, -1.0796,  0.0168],
        [-0.0914,  1.0674,  0.1128,  ...,  1.5370, -0.7021,  0.4136],
        [-0.4389,  0.1975,  0.1769,  ...,  1.2585, -0.9988,  0.9610],
        ...,
        [-0.5434,  0.4785,  0.2987,  ...,  1.2145, -1.3875,  0.4589],
        [-0.4068,  0.6838,  0.2546,  ...,  0.3009, -1.6419,  0.1866],
        [-0.6097,  1.0765,  0.8175,  ...,  1.0235, -2.0095,  0.1364]])
tensor([5.4609e-01, 8.4666e-04, 9.8479e-01, 7.5767e-04, 3.4211e-04, 8.6509e-04,
        9.6488e-04, 5.3742e-04])


In [40]:
activation = {}
dataloader8 = DataLoader(dataset, batch_size=8, shuffle=False)

outputs8 = torch.Tensor()
for data in iter(dataloader8):
    with torch.no_grad():
        x, y = data
        inputs = []
        for n in range(len(x)):
            inputs.append((y[n], x[n]))

        out = my_model(inputs).squeeze().detach().cpu()
        outputs8 = torch.cat((outputs8, out), dim=0)

embeds8 = activation['encoder']
print(embeds8[0][0,:,:])
features8 = activation['classifier']
print(features8[0][0,:,:])
print(outputs8)

tensor([4.9472e-01, 9.0526e-04, 9.9266e-01, 1.1670e-03, 3.0970e-04, 5.2615e-04,
        6.9786e-04, 4.8283e-04])
tensor([[ 0.1924,  0.5990, -0.0358,  ...,  0.9546, -0.1360, -0.3861],
        [ 0.1858,  0.0975, -0.1853,  ...,  0.3884, -0.1616, -0.1158],
        [ 0.0609, -0.2148, -0.2203,  ...,  0.0230,  0.0243,  0.0387],
        ...,
        [ 0.1819, -0.3224,  0.0600,  ...,  0.1130, -0.1211, -0.0456],
        [ 0.1992, -0.4265,  0.0680,  ...,  0.1228, -0.1319, -0.0453],
        [ 0.1975, -0.3415,  0.1055,  ...,  0.0735, -0.1216, -0.0568]])
tensor([[ 0.4855,  0.1922, -0.5461,  ...,  1.2570,  0.2259, -0.3514],
        [ 0.5211, -0.3332, -0.8597,  ...,  0.8097,  0.1515, -0.2381],
        [ 0.1744, -0.8706, -0.9513,  ...,  0.1007,  0.6794, -0.0963],
        ...,
        [ 0.3365, -0.7977, -0.6612,  ...,  0.1568,  0.4583, -0.2400],
        [ 0.3428, -0.8779, -0.6439,  ...,  0.1722,  0.4450, -0.2218],
        [ 0.3567, -0.7939, -0.5999,  ...,  0.1324,  0.4521, -0.2219]])
